In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pickle
from statsmodels.tsa.vector_ar.var_model import VAR

In [13]:
csv_dataset = '../data/processed/data.csv'
csv = pd.read_csv(csv_dataset, encoding='utf_8')
df = pd.DataFrame(csv)
df = df.drop(columns='date')

In [14]:
lags = 72

# Split the data into training and testing sets
train = df[:-lags]
test = df[-lags:]

# Fit a VAR model with 72 lags
model = VAR(train)
results = model.fit(lags)

pickle.dump(model, open('../models/var', 'wb'))

In [15]:
predictions = results.forecast(test.values, steps=lags)

# Convert the predictions back to a DataFrame
predictions = pd.DataFrame(predictions, columns=df.columns, index=test.index)

predictions_train = results.fittedvalues
train = train[lags:]

with open('../reports/train_metrics.txt', 'w') as file:
    file.write('MAE:' + str(metrics.mean_absolute_error(train, predictions_train)) + '\n')
    file.write('MSE:' + str(metrics.mean_squared_error(train, predictions_train)) + '\n')
    file.write('EVS:' + str(metrics.explained_variance_score(train, predictions_train)) + '\n')

In [16]:
with open('../reports/metrics.txt', 'w') as file:
    file.write('MAE:' + str(metrics.mean_absolute_error(test, predictions)) + '\n')
    file.write('MSE:' + str(metrics.mean_squared_error(test, predictions)) + '\n')
    file.write('EVS:' + str(metrics.explained_variance_score(test, predictions)) + '\n')
